In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

from tests.sama4d.video_to_tracks.keypoint_tracking_task import KeypointTrackingTask
from tests.sama4d.data_curation import get_cheezitbox_scene_loader
import rerun as rr

In [4]:
scene_spec = get_cheezitbox_scene_loader()
task = KeypointTrackingTask(scene_spec["feature_track_data_loader"], scene_name=scene_spec["scene_name"], n_frames=10)

In [5]:
rr.init(f'task--{task.name}-12')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [37]:
import b3d.chisight.particle_system as ps
import b3d.chisight.particle_system_patch_tracking as pt
import b3d.chisight.dense.likelihoods as likelihoods
from b3d.chisight.dense.differentiable_renderer import DifferentiableRendererHyperparams
from b3d import Pose
import b3d
import genjax
from genjax import ChoiceMapBuilder as C
import jax
import jax.numpy as jnp
genjax.pretty()

In [18]:
task_spec = task.get_task_specification()
video = task_spec["video"]
assert video.shape[-1] == 4, "Expected video to be RGBD"
poses_WC = task_spec["poses_WC"]
initial_patch_centers_2D = task_spec["initial_keypoint_positions_2D"]
r = task_spec["renderer"]
fx, fy, cx, cy = r.fx, r.fy, r.cx, r.cy
renderer : b3d.Renderer = r

In [10]:
(patches, poses_CP_at_time0, patch_points_C) = pt.get_patches(
    initial_patch_centers_2D, video, Pose.identity(), fx, fy, cx, cy
)

In [38]:
depth_scale, color_scale, mindepth, maxdepth = 0.0001, 0.002, -20.0, 20.0
likelihood = likelihoods.get_uniform_multilaplace_image_dist_with_fixed_params(
        renderer.height, renderer.width, depth_scale, color_scale, mindepth, maxdepth
)
@genjax.gen
def wrapped_likelihood(mesh : b3d.Mesh, args):
    weights, attributes = b3d.chisight.dense.differentiable_renderer.render_to_rgbd_dist_params(
        renderer, mesh.vertices, mesh.faces, mesh.vertex_attributes,
        DifferentiableRendererHyperparams(3, 1e-5, 1e-2, -1)
    )
    obs = likelihood(weights, attributes) @ "image"
    return obs, {"diffrend_output": (weights, attributes)}

model = ps.make_dense_gps_model(wrapped_likelihood)

In [39]:
max_num_timesteps = genjax.Pytree.const(20)
num_particles = genjax.Pytree.const(patches.vertices.shape[0])
num_clusters = num_particles
relative_particle_poses_prior_params = (Pose.identity(), .5, 0.25)
initial_object_poses_prior_params = (Pose.identity(), 2., 0.5)
camera_pose_prior_params = (Pose.identity(), 0.1, 0.1)
instrinsics = genjax.Pytree.const(renderer.get_intrinsics_object())
sigma_obs = 0.2

model_args = (
    patches,
    (
        max_num_timesteps, # const object
        1,
        num_particles, # const object
        num_clusters, # const object
        relative_particle_poses_prior_params,
        initial_object_poses_prior_params,
        camera_pose_prior_params
    ),
    ()
)

In [40]:
image = video[0]
particle_poses = jax.tree.map(
    lambda arr: jnp.tile(arr, (num_particles.const, 1)),
    Pose.identity()
)
pose_WC = poses_WC[0]
object_assignments = jnp.arange(num_particles.const)
object_poses = poses_CP_at_time0
vis_mask = jnp.ones((num_particles.const,), dtype=bool)

dc = C["state0"].set(C.d({
    "particle_poses": particle_poses,
    "object_assignments": object_assignments,
    "object_poses": object_poses,
    "initial_camera_pose": pose_WC,
    "initial_visibility": vis_mask,
}))
dc

StaticChm(...)

In [41]:
constraints = C.d({
    "particle_dynamics": dc,
    "obs": C["image"].set(image),
})

constraints

XorChm(...)

In [42]:
trace = model.importance(
    jax.random.PRNGKey(200),
    constraints,
    model_args
)

TypeError: Method[1m[35m genjax._src.generative_functions.combinators.scan.ScanCombinator.update_change_target()[0m parameter [1m[34margdiffs[0m=[1m[31m(Mask(...), Mask(...))[0m violates type hint [1m[32mtyping.Annotated[tuple, Is[lambda v: Diff.static_check_tree_diff(v)]][0m, as [1m[33mtuple [0m[1m[31m(Mask(...), Mask(...))[0m violates validator Is[lambda v: Diff.static_check_tree_diff(v)]:
    False == Is[lambda v: Diff.static_check_tree_diff(v)].

In [43]:
simtr = model.simulate(jax.random.PRNGKey(20), model_args)
simtr

StaticTrace(...)